In [22]:
from utils.chat import OllamaChat
from utils.prompts import get_persona

In [23]:
system_prompt = get_persona("example")


In [24]:
chat = OllamaChat(
        model='llama3.2',  # Replace with your desired model
        system_prompt=system_prompt,
        n_ctx=4096,
        temperature=0.7
    )

In [32]:
result=chat.chat(user_prompt="yeah sure")
# chat._save_conversation()

In [33]:
print(result)

We've worked with several clients in various industries, and I'd be happy to share some success stories with you.

For example, one of our e-commerce clients saw a 40% reduction in response times after implementing ServiceFlow AI. They also reported a significant increase in customer satisfaction scores, which ultimately led to increased sales and revenue.

Another client in the financial services sector used ChatGenius Pro to automate their chatbot conversations. By integrating it with their existing CRM system, they were able to improve their response time by 30% and reduce the number of manual tasks required to manage customer inquiries.

We've also had success with InsightHub Analytics, which provides real-time analytics and reporting capabilities for our clients. It helps them gain valuable insights into their customer behavior and sentiment, enabling them to make data-driven decisions that drive business growth.

I'd love to share more about these success stories and how they can

In [27]:
chat._load_conversation()

In [42]:
result=chatbot.chat(message="What Services Do you provide ?")